<a href="https://colab.research.google.com/github/menhguin/natural_language_rl/blob/main/Goodfire_LLM_Eval_Harness_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1. Install EleutherAI Evaluations Harness
*   Logging into WandB is optional.
*   Logging into Huggingface API is required to run GPQA. This is to prevent database leakage.
*   Uses Goodfire API! Experimental as of Jan 2025

In [1]:
import os
import huggingface_hub
from google.colab import userdata

# Install latest versions of necessary libraries
!pip install goodfire
!pip install -e git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm] # skip if you don't want to use wandb to log results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: i

Automated login for Hugging Face Hub via Colab Secrets. If you don't have this, it'll prompt for manual login if you don't have one. If you completely remove this, you can't run GPQA or use Llama models via HF.

In [2]:
# Check for Huggingface API key and log in if available, otherwise prompt for manual login
hf_token = userdata.get('HF_READ_TOKEN')
if hf_token:
    huggingface_hub.login(hf_token)
else:
    print("Huggingface token not found. Please login manually.")
    !huggingface-cli login

Automated login for WandB via Colab Secrets. If you don't have this, it'll just prompt you later if you use wandb_args.

In [3]:
# Check for WandB API key and log in if available, otherwise skip login
wandb_token = userdata.get('WANDB_API_KEY')
if wandb_token:
    os.environ["WANDB_API_KEY"] = wandb_token
    import wandb
    wandb.login()
else:
    print("WandB token not found. Continuing without logging into WandB.")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: minh1228 (menhguin). Use `wandb login --relogin` to force relogin


Automated login for Goodfire API via Colab Secrets.

In [4]:
# Get API key from Colab secrets
GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')
if not GOODFIRE_API_KEY:
    raise ValueError("Please set GOODFIRE_API_KEY in Colab secrets")
os.environ['GOODFIRE_API_KEY'] = GOODFIRE_API_KEY

#Step 2. Run evaluation

In [7]:
!python -m lm_eval \
    --model goodfire \
    --model_args pretrained=meta-llama/Meta-Llama-3.1-8B-Instruct \
    --tasks gsm8k_cot_llama \
    --num_fewshot 8 \
    --limit 30 \
    --apply_chat_template \
    --fewshot_as_multiturn \
    --batch_size auto \
    --log_samples \
    --output_path ./lm-eval-output/ \
    --gen_kwargs top_p=0.9,temperature=1.0

2025-01-11 17:50:26.904809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-11 17:50:26.928468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-11 17:50:26.935370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 17:50:26.952194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-11 17:50:28.356190: W tensorflow/compiler/tf2

#Reference: EleutherAI Eval Harness task list
For those curious to run other evals! Please note that Min P is currently only accessible for `generate_until` tasks. There is currently no easy way to index these tasks, I just Ctrl + F'd `generate_until` on the [EleutherAI Evals Harness Repo](https://github.com/EleutherAI/lm-evaluation-harness).

In [6]:
# Test Goodfire Client
import goodfire

client = goodfire.Client(api_key=GOODFIRE_API_KEY)

# Simple test call
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "Say hello!"}],
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    max_completion_tokens=10
)

# Access response using ChatCompletion object attributes
print("Test response:", response.choices[0].message['content'])

Test response: Hello! It's nice to meet you. How


In [ ]:
 !lm-eval --tasks list